In [1]:
import sys
sys.path.append('..')

In [2]:
import os

def all_records_in(path):
    """Find all records in a record directory.
    The given directory must contain no other files!
    """
    return list(set([f.split('.')[0] for f in os.listdir(path)]))

In [3]:
conf = {
    "input_dir": "../data/noisy3",
    "records": all_records_in("../data/noisy3"),
#     "records": ["{}_em_3".format(num) for num in range(100,110)],
    "output_dir": "./generated/dropout",
    "save_annotations": False,
    "trigger_distance": 2,
    "save_model": False,
    "sampto": 5000,
    "verbose": False,
    "detectors": [
        {
            "type": "XiangDetector",
            "name": "XiangZeroDrop",
            "batch_size": 32,
            "window_size": 90,
            "detection_size": 30,
            "threshold": 0.9,
            "tolerance": 10,
            "aux_ratio": 5,
            "epochs": 3
        },
#         {
#             "type": "RXDetector",
#             "name": "RXHalfInputDrop",
#             "batch_size": 32,
#             "window_size": 90,
#             "detection_size": 30,
#             "input_dropout": 0.5,
#             "threshold": 0.9,
#             "tolerance": 10,
#             "aux_ratio": 5,
#             "epochs": 3
#         },
#         {
#             "type": "RXDetector",
#             "name": "RXHalfConvDrop",
#             "batch_size": 32,
#             "window_size": 90,
#             "detection_size": 30,
#             "conv_dropout": 0.5,
#             "threshold": 0.9,
#             "tolerance": 10,
#             "aux_ratio": 5,
#             "epochs": 3
#         },
#         { !!!
#             "type": "RXDetector",
#             "name": "RXQuarterInputDrop",
#             "batch_size": 32,
#             "window_size": 90,
#             "detection_size": 30,
#             "input_dropout": 0.25,
#             "threshold": 0.9,
#             "tolerance": 10,
#             "aux_ratio": 5,
#             "epochs": 3
#         },
#         {
#             "type": "RXDetector",
#             "name": "RXQuarterConvDrop",
#             "batch_size": 32,
#             "window_size": 90,
#             "detection_size": 30,
#             "conv_dropout": 0.25,
#             "threshold": 0.9,
#             "tolerance": 10,
#             "aux_ratio": 5,
#             "epochs": 3
#         },
#         {
#             "type": "RXDetector",
#             "name": "RXQuarterInputAndConvDrop",
#             "batch_size": 32,
#             "window_size": 90,
#             "detection_size": 30,
#             "input_dropout": 0.25,
#             "conv_dropout": 0.25,
#             "threshold": 0.9,
#             "tolerance": 10,
#             "aux_ratio": 5,
#             "epochs": 3
#         },
        {
            "type": "RXDetector",
            "name": "RXHalfInputAndConvDrop",
            "batch_size": 32,
            "window_size": 90,
            "detection_size": 30,
            "input_dropout": 0.5,
            "conv_dropout": 0.5,
            "threshold": 0.9,
            "tolerance": 10,
            "aux_ratio": 5,
            "epochs": 3
        }
    ]
}

In [4]:
from raccoon.utils.builders import evaluator_from_dict

evaluator = evaluator_from_dict(conf)

Using TensorFlow backend.


In [5]:
reports = evaluator.loocv()

Epoch 1/3
7211/7211 [==============================] - 19s 3ms/step - loss: 0.0967
Epoch 2/3
7211/7211 [==============================] - 19s 3ms/step - loss: 0.0838
Epoch 3/3
7211/7211 [==============================] - 16s 2ms/step - loss: 0.0854
Epoch 1/3
7211/7211 [==============================] - 19s 3ms/step - loss: 0.1076
Epoch 2/3
Epoch 1/3
7211/7211 [==============================] - 18s 2ms/step - loss: 0.0855
Epoch 3/3
7211/7211 [==============================] - 15s 2ms/step - loss: 0.0838
Epoch 1/3
7211/7211 [==============================] - 19s 3ms/step - loss: 0.1082
Epoch 2/3
Epoch 1/3
7211/7211 [==============================] - 18s 3ms/step - loss: 0.0845
Epoch 3/3
7211/7211 [==============================] - 21s 3ms/step - loss: 0.0842
Epoch 1/3
7211/7211 [==============================] - 17s 2ms/step - loss: 0.1020
Epoch 2/3

7211/7211 [==============================] - 15s 2ms/step - loss: 0.0870
Epoch 3/3
7211/7211 [==============================] - 15s 2ms/ste

7211/7211 [==============================] - 23s 3ms/step - loss: 0.1043
Epoch 2/3
Epoch 1/3
7211/7211 [==============================] - 22s 3ms/step - loss: 0.0860
Epoch 3/3
7211/7211 [==============================] - 22s 3ms/step - loss: 0.0834
Epoch 1/3
7211/7211 [==============================] - 21s 3ms/step - loss: 0.1039
Epoch 2/3
Epoch 1/3
7211/7211 [==============================] - 19s 3ms/step - loss: 0.0841
Epoch 3/3
7211/7211 [==============================] - 19s 3ms/step - loss: 0.0788
Epoch 1/3
7211/7211 [==============================] - 21s 3ms/step - loss: 0.0985
Epoch 2/3
Epoch 1/3
7211/7211 [==============================] - 20s 3ms/step - loss: 0.0827
Epoch 3/3
7211/7211 [==============================] - 22s 3ms/step - loss: 0.0842
Epoch 1/3
7211/7211 [==============================] - 66s 9ms/step - loss: 0.1045
Epoch 2/3
Epoch 1/3
7211/7211 [==============================] - 26s 4ms/step - loss: 0.0849
Epoch 3/3
7211/7211 [==============================] - 24

Process ForkPoolWorker-3628:
Process ForkPoolWorker-3627:
Process ForkPoolWorker-3626:
Process ForkPoolWorker-3630:
Process ForkPoolWorker-3618:
Process ForkPoolWorker-3631:
Process ForkPoolWorker-3624:
Process ForkPoolWorker-3625:
Process ForkPoolWorker-3623:
Process ForkPoolWorker-3622:
Process ForkPoolWorker-3632:
Process ForkPoolWorker-3620:
Process ForkPoolWorker-3621:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-3619:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceb

KeyboardInterrupt: 

In [ ]:
acc_reports = [report for report in reports if type(report['Test Record']) is list]
acc_reports

In [ ]:
import numpy as np

metrics = ('Se', 'PPV', 'F1')
detectors = list(set([ar['Detector'] for ar in acc_reports]))

ses = [
    [ar['Sensitivity'] for ar in acc_reports if ar['Detector'] == detector]
    for detector in detectors]
ppvs = [
    [ar['PPV'] for ar in acc_reports if ar['Detector'] == detector]
    for detector in detectors]
f1s = [
    [ar['F1'] for ar in acc_reports if ar['Detector'] == detector]
    for detector in detectors]
    
means = np.array([np.nanmean(ses, axis=1), np.nanmean(ppvs, axis=1), np.nanmean(f1s, axis=1)]).T
stds = np.array([np.nanstd(ses, axis=1), np.nanstd(ppvs, axis=1), np.nanstd(f1s, axis=1)]).T

In [ ]:
print(ppvs)

In [ ]:
from scipy.stats import ttest_ind

print("Se : {}".format(ttest_ind(ses[0], ses[2], nan_policy='omit')))
print("PPV: {}".format(ttest_ind(ppvs[0], ppvs[2], nan_policy='omit')))
print("F1 : {}".format(ttest_ind(f1s[0], f1s[2], nan_policy='omit')))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({'font.size': 14})

fig, ax = plt.subplots()
fig.set_size_inches(10, 5)

bar_width = 0.6 / len(detectors)
index = np.arange(3) # Se, PPV, F1

for ax_num, (detector, mean, std) in enumerate(zip(detectors, means, stds)):
    ax.bar(index+ax_num*bar_width, mean, bar_width, yerr=std, label=detector)

ax.set_xticks(index + 0.5*(len(detectors)-1)*bar_width)
ax.set_xticklabels(metrics)
ax.set_ylim(0, 1)

ax.legend(bbox_to_anchor=(1.05, 1))

plt.show()